# PRÁCTICA GUIADA 02

---
**Curso:** Data Mining Tools  
**Semestre:** 2025-2  
**Docente:** Carlos Fernando Montoya Cubas  
**Grupo:** 08  

---


## ALUMNOS
- Mildred Micaela Marchan Quispe  
- Rody Sebastian Vilchez Marin  
- Rosa Maria Rodríguez Valencia

## Instrucciones:  

Dado el dataset de fraude crediticio disponible en:   
https://www.kaggle.com/datasets/mishra5001/credit-card/data  
1. Realizar el preprocesamiento de variables y examinar las relaciones entre los 
datos.  
 
2. Realizar el escalamiento multidimensional de los datos y realizar la 
interpretación correspondiente.  
 
3. Crear una clase en python que reciba como parámetro el dataset y los  
transformadores correspondientes y pueda realizar el preprocesamiento de las 
variables. La clase debe interpretar la función fit que calibra los  
transformadores y la función transform que realiza la transformación  
propiamente dicha. 


In [ ]:
!pip install pandas matplotlib seaborn scikit-learn wordcloud -q

In [ ]:

# paquete que implementa varios métodos de codificación de variables categóricas
!pip install category_encoders kaggle -q


In [ ]:

## Cargando las bibliotecas
# importa  pandas
import pandas as pd

# gráficos
import matplotlib.pyplot as plt
import seaborn as sns

# Escalonamento multidmensional
from sklearn.manifold import MDS

# transformacion de atributos
from category_encoders import OneHotEncoder, OrdinalEncoder

# cálculo de distancias
from scipy.spatial.distance import pdist, squareform



## Base de datos

**Credit Card Fraud Detection**

Este dataset busca obtener insights sobre clientes que incumplen pagos de tarjetas de crédito, usando atributos como Income_Total, AMT_APPLICATION, AMT_CREDIT y más de 120 variables.

Incluye también el Previous Application Data Set, que permite analizar patrones y variaciones considerando el historial crediticio.

Fue tomado como parte de una asignación académica, aplicando EDA para identificar tendencias y riesgos de incumplimiento. [aqui](https://www.kaggle.com/datasets/mishra5001/credit-card/data)


In [ ]:

!kaggle datasets download -d mishra5001/credit-card # setear kaggle api

!mkdir -p data
!unzip -o credit-card.zip -d data # comprimir en data
!rm credit-card.zip # borrar zip


In [ ]:

APP = pd.read_csv('data/application_data.csv')
print(APP.shape)
APP.head(2)

In [ ]:

PREVIOUS = pd.read_csv('data/previous_application.csv')
print(PREVIOUS.shape)
PREVIOUS.head(2)


In [ ]:
COLUMNS = pd.read_csv('data/columns_description.csv',sep=",", encoding='cp1252', index_col=0)
print(COLUMNS.shape)
COLUMNS.head(2)


### Seleccion del dataset

Los datos de la aplicación principal se encuentran en el archivo `application_data.csv`, que contiene 122 columnas y 307511 filas. El conjunto de datos de aplicaciones previas está en `previous_application.csv`, con 37 columnas y 1670214 filas. La descripción de las columnas está en `columns_description.csv`.

Exploraremos `columns_description.csv` para entender las variables, y luego escogeremos un conjunto de datos para análisis y preprocesamiento.


In [ ]:


COLUMNS["Table"].value_counts()

In [ ]:

COLUMNS[COLUMNS['Table'] == 'application_data'].describe().T

In [ ]:

COLUMNS[COLUMNS['Table'] == 'previous_application.csv'].describe().T




Tratamiento`Special` segun tipo de tabla`Table`


In [ ]:

dfs = COLUMNS['Table'].unique()

for df in dfs:
    prev_app = COLUMNS[COLUMNS['Table'] == df]
    freq = prev_app['Special'].value_counts(normalize=True, dropna=False) * 100
    freq = freq.round(2)
    print(df, end="\n\n")
    
    print(freq)
    print("\n\n")


In [ ]:

# wordclod por tratamiento `Special` por tipo de tabla`Table`
from wordcloud import WordCloud

text_prev = ' '.join(COLUMNS[COLUMNS['Table'] == 'previous_application.csv']['Description'].dropna().values)
text_app = ' '.join(COLUMNS[COLUMNS['Table'] == 'application_data']['Description'].dropna().values)


wordcloud_prev = WordCloud(width=800, height=400, background_color='white').generate(text_prev)
wordcloud_app = WordCloud(width=800, height=400, background_color='white').generate(text_app)

plt.figure(figsize=(20, 16))

plt.subplot(121)
plt.imshow(wordcloud_prev, interpolation='bilinear')
plt.title('Previous Application Description')
plt.axis('off')

plt.subplot(122)
plt.imshow(wordcloud_app, interpolation='bilinear')
plt.title('Application Data Description')
plt.axis('off')
plt.show()



Dado que el proposito de la leccion es limpiar datos, se opta por trabajar con el dataset `previoys_application.csv` que contiene los datos crudos. Este conjunto es el mas adecuado para aplicar y practicar tecnicas de preprocesamiento y analisis exploratorio.

El dataset de `application_data.csv` se podra llegar a usar como control


In [ ]:

# descripcion de los atributos numéricos
PREVIOUS.describe().T

In [ ]:

# descripcion de los atributos categóricos
PREVIOUS.describe(include = 'object').T

In [ ]:
PREVIOUS.isnull().mean().sort_values(ascending=False).__round__(3).head(5)

In [ ]:
#sns.heatmap(PREVIOUS.isnull(), cbar=False) # demora 2 dias

Vamos a analizar los atributos categóricos, para clasificarlos entre discreto y ordinal


In [ ]:

for column_name in PREVIOUS.select_dtypes(include=["object_"]):
  print(column_name, "->", PREVIOUS[column_name].unique())
  print()
  


Vamos a separar los atributos nominales y ordinales para hacer una transformacion a numérica, para calcular distancias:
    


In [ ]:
nominal_cols = ['NAME_CONTRACT_TYPE', 'NAME_CASH_LOAN_PURPOSE', 
                'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 
                'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO',
                'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE','NAME_SELLER_INDUSTRY',
                'PRODUCT_COMBINATION', 'FLAG_LAST_APPL_PER_CONTRACT',  'NAME_CONTRACT_STATUS'
                ]

ordinal_cols = ['WEEKDAY_APPR_PROCESS_START', 
               'NAME_YIELD_GROUP'
                ]

In [ ]:
mappings = [
    {'col': 'WEEKDAY_APPR_PROCESS_START',
     'mapping': {
         'MONDAY': 0,
         'TUESDAY': 1,
         'WEDNESDAY': 2,
         'THURSDAY': 3,
         'FRIDAY': 4,
         'SATURDAY': 5,
         'SUNDAY': 6
     }},
    
    {'col': 'NAME_YIELD_GROUP',
     'mapping': {
         'low_action': 0,
         'low_normal': 1,
         'middle': 2,
         'high': 3,
         'XNA': -1
     }}
]



Precisamos explicitar a ordem para os atributos ordinais, uma vez que a ordem em que eles aparacem pode ser diferente da ordem desejada:
    


Ahora vamos a crear los objetos que hacen la transformación. Para los atributos nominales, utilizaremos la codificación 1 de m (One-Hot-Encoding). En cuanto a los atributos categóricos, usemos el orden creado anteriormente

In [ ]:
onehotenc = OneHotEncoder(cols = nominal_cols)
ordinalenc = OrdinalEncoder(cols = ordinal_cols, mapping = mappings)

Inicialmente, transformamos os atributos nominales:

In [ ]:
PREVIOUS_1 = onehotenc.fit_transform(PREVIOUS)

In [ ]:
PREVIOUS_1.head()

Ahora aplicamos la transformacion de los ordinales:

In [ ]:
PREVIOUS_2 = ordinalenc.fit_transform(PREVIOUS_1)

In [ ]:
PREVIOUS_2

Finalmente, un tratamento de valores ausentes

In [ ]:
PREVIOUS_3 = PREVIOUS_2.fillna(PREVIOUS_2.median())

In [ ]:
PREVIOUS_3.isnull().sum().sum()

Para acelerar los cálculos, tomemos una muestra aleatoria de esta versión procesada. Además, para que un atributo no tenga mayor importancia que los demás, cambiemos la escala para que todos tengan valores entre 0 y 1:

In [ ]:

sample = PREVIOUS_3.sample(1000)
print(sample.columns.to_list())

max_amt_credit = max(sample.AMT_CREDIT)
max_rate_down_payment = max(sample.RATE_DOWN_PAYMENT)

# normalización Min-Max
sample = sample.loc[:, sample.nunique() > 1]
sample = (sample - sample.min()) / (sample.max() - sample.min()) # evita columnas constantes


In [ ]:
sample.columns

Ahora, vamos a calcular la matriz de distancias



In [ ]:
dist = squareform(pdist(sample,'euclidean'))

In [ ]:
dist

Ahora vamos a crear un objeto Método que calcule el escalado multidimensional. Como ya hemos calculado la matriz de disimilitud, pasamos el argumento 'precalculado' (si no lo hacemos, el método calcula internamente la matriz de distancia euclidiana). La implementación utiliza un enfoque iterativo para minimizar la función de estrés. La opción detallada es para que sigamos la evolución de esta función.

In [ ]:
mds = MDS(dissimilarity='precomputed',verbose=2)

Una vez que se ha creado el objeto, podemos aplicarlo a la matriz de distancia, que calculamos anteriormente.

In [ ]:
transformed = mds.fit_transform(dist)

In [ ]:
dist = squareform(pdist(sample,'cityblock'))
transformed_city = mds.fit_transform(dist)


Para visualizar esta transformación, podemos hacer un diagrama de dispersión. Para ayudar a la interpretación, utilizaremos un código de colores basado en la edad de los individuos:

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

scatter1 = ax1.scatter(transformed[:,0], transformed[:,1], 
                       c=sample.AMT_CREDIT * max_amt_credit, cmap="viridis")
legend1 = ax1.legend(*scatter1.legend_elements(), title="Amount Credit (Euclidean)")
ax1.add_artist(legend1)

scatter2 = ax2.scatter(transformed_city[:,0], transformed_city[:,1], 
                       c=sample.AMT_CREDIT * max_amt_credit, cmap="viridis")
legend2 = ax2.legend(*scatter2.legend_elements(), title="Amount Credit (Cityblock)")
ax2.add_artist(legend2)

plt.show()


In [ ]:

import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

scatter1 = ax1.scatter(transformed[:,0], transformed[:,1], 
                       c=sample.NAME_YIELD_GROUP , cmap="viridis")
legend1 = ax1.legend(*scatter1.legend_elements(), title="Name Yield Group (Euclidean)")
ax1.add_artist(legend1)

scatter2 = ax2.scatter(transformed_city[:,0], transformed_city[:,1], 
                       c=sample.NAME_YIELD_GROUP, cmap="viridis")
legend2 = ax2.legend(*scatter2.legend_elements(), title="Name Yield Group (Cityblock)")
ax2.add_artist(legend2)

plt.show()

In [ ]:

import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

scatter1 = ax1.scatter(transformed[:,0], transformed[:,1], 
                       c=sample.NAME_CONTRACT_TYPE_1 , cmap="viridis")
legend1 = ax1.legend(*scatter1.legend_elements(), title="Name Contract Type (Euclidean)")
ax1.add_artist(legend1)

scatter2 = ax2.scatter(transformed_city[:,0], transformed_city[:,1], 
                       c=sample.NAME_CONTRACT_TYPE_1, cmap="viridis")
legend2 = ax2.legend(*scatter2.legend_elements(), title="Name Contract Type  (Cityblock)")
ax2.add_artist(legend2)

plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

scatter1 = ax1.scatter(transformed[:,0], transformed[:,1], 
                       c=sample.RATE_DOWN_PAYMENT * max_rate_down_payment, cmap="viridis")
legend1 = ax1.legend(*scatter1.legend_elements(), title="Rate Down Payment (Euclidean)")
ax1.add_artist(legend1)

scatter2 = ax2.scatter(transformed_city[:,0], transformed_city[:,1], 
                       c=sample.RATE_DOWN_PAYMENT * max_rate_down_payment, cmap="viridis")
legend2 = ax2.legend(*scatter2.legend_elements(), title="Rate Down Payment (Cityblock)")
ax2.add_artist(legend2)

plt.show()


In [ ]:

import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

scatter1 = ax1.scatter(transformed[:,0], transformed[:,1], 
                       c=sample.NAME_CLIENT_TYPE_2 , cmap="viridis")
legend1 = ax1.legend(*scatter1.legend_elements(), title="Name Client Type 2(New) (Euclidean)")
ax1.add_artist(legend1)

scatter2 = ax2.scatter(transformed_city[:,0], transformed_city[:,1], 
                       c=sample.NAME_CLIENT_TYPE_2, cmap="viridis")
legend2 = ax2.legend(*scatter2.legend_elements(), title="Name Client Type 2 (New) (Cityblock)")
ax2.add_artist(legend2)

plt.show()

## Asociaciones entre variables

Ahora investiguemos algunas asociaciones entre variables en esta base (usando la base original)

In [ ]:

plt.figure(figsize=(10,6))
sns.boxplot(x=PREVIOUS['NAME_CONTRACT_TYPE'], y=PREVIOUS['AMT_CREDIT'])
plt.title('Distribución de AMT_CREDIT según NAME_CONTRACT_TYPE')
plt.show()
    


In [ ]:

# Asociación entre dos variables categóricas
pd.crosstab(PREVIOUS['NAME_CONTRACT_TYPE'], PREVIOUS['NAME_CLIENT_TYPE'], margins=True, normalize='index').round(3)


In [ ]:

plt.figure(figsize=(8,6))
sns.scatterplot(x=PREVIOUS_3['AMT_CREDIT'], y=PREVIOUS_3['RATE_DOWN_PAYMENT'])
plt.title('Relación entre AMT_CREDIT y RATE_DOWN_PAYMENT')
plt.xlabel('AMT_CREDIT')
plt.ylabel('RATE_DOWN_PAYMENT')
plt.show()

In [ ]:
    # 1. Preprocesamiento y exploración de relaciones
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import MDS
from category_encoders import OneHotEncoder, OrdinalEncoder


In [ ]:

df = PREVIOUS.copy()  

df_num = df.select_dtypes(include=[np.number])
df_cat = df.select_dtypes(include=['object', 'category'])

df_num = df_num.fillna(df_num.median())
df_cat = df_cat.fillna('Desconocido')

# Codificación de variables categóricas
onehot = OneHotEncoder(cols=df_cat.columns, use_cat_names=True)
df_cat_enc = onehot.fit_transform(df_cat)

# Unir datos preprocesados
df_proc = pd.concat([df_num, df_cat_enc], axis=1)

# Análisis exploratorio: correlación y distribuciones
plt.figure(figsize=(10,6))

cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.set(rc={'figure.figsize':(15,15)})
corr = df_num.corr()
sns.heatmap(corr, cmap=cmap, vmax=1, vmin=-1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.title('Matriz de correlación de variables numéricas')
plt.show()


# Distribuciones
# for col in df_num.columns:
#     plt.figure()
#     sns.histplot(df_num[col], kde=True)
#     plt.title(f'Distribución de {col}')
#     plt.show()

# 2. Escalamiento multidimensional (MDS)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_proc)
mds = MDS(n_components=2, random_state=42)
mds_coords = mds.fit_transform(df_scaled)

plt.figure(figsize=(8,6))
plt.scatter(mds_coords[:,0], mds_coords[:,1], alpha=0.7)
plt.title('Escalamiento Multidimensional (MDS)')
plt.xlabel('Componente 1')
plt.ylabel('Componente 2')
plt.show()

# 3. Clase de preprocesamiento generalizada
from sklearn.base import BaseEstimator, TransformerMixin

class GeneralPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, num_strategy='median', cat_strategy='Desconocido', encoder=None, scaler=None):
        self.num_strategy = num_strategy
        self.cat_strategy = cat_strategy
        self.encoder = encoder if encoder is not None else OneHotEncoder(use_cat_names=True)
        self.scaler = scaler if scaler is not None else StandardScaler()
        self.num_cols = None
        self.cat_cols = None
    
    def fit(self, X, y=None):
        self.num_cols = X.select_dtypes(include=[np.number]).columns
        self.cat_cols = X.select_dtypes(include=['object', 'category']).columns
        self.num_medians = X[self.num_cols].median()
        self.encoder.fit(X[self.cat_cols].fillna(self.cat_strategy))
        self.scaler.fit(pd.concat([
            X[self.num_cols].fillna(self.num_strategy),
            self.encoder.transform(X[self.cat_cols].fillna(self.cat_strategy))
        ], axis=1))
        return self
    
    def transform(self, X):
        X_num = X[self.num_cols].fillna(self.num_strategy)
        X_cat = X[self.cat_cols].fillna(self.cat_strategy)
        X_cat_enc = self.encoder.transform(X_cat)
        X_proc = pd.concat([X_num, X_cat_enc], axis=1)
        X_scaled = self.scaler.transform(X_proc)
        return X_scaled




In [ ]:
preproc = GeneralPreprocessor()
preproc.fit(df)
datos_transformados = preproc.transform(df)